# TAXI_NY

In [1]:
%load_ext autoreload
%autoreload 2

## Import des librairies

In [2]:
import split
from distance import compute_distance

import time
import pandas as pd
import numpy as np

import matplotlib as plt

import seaborn as sns

import xgboost as xgb

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

## Chargement des données

In [3]:
splitter = split.Splitter("data",",")
splitter.extract_data("train.csv","key","annee")

Fichiers déjà splitté.


In [4]:
global_train_df = pd.read_csv("data/CURATED/annee/2015.csv", header=0)
global_test_df = pd.read_csv("data/RAW/test.csv", header=0)

In [5]:
train_df = global_train_df.copy()
test_df = global_test_df.copy()

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3856809 entries, 0 to 3856808
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 235.4+ MB


## TRAIN. Nettoyage des données 

In [7]:
train_df_clean = train_df[(train_df["fare_amount"] > 0)]
train_df_clean = train_df_clean[(train_df_clean["passenger_count"] > 0)]
train_df_clean = train_df_clean[(train_df_clean["passenger_count"] < 7)]

train_df_clean = train_df_clean[(train_df_clean["pickup_longitude"] != 0)]
train_df_clean = train_df_clean[(train_df_clean["pickup_latitude"] != 0)]
train_df_clean = train_df_clean[(train_df_clean["dropoff_longitude"] != 0)]
train_df_clean = train_df_clean[(train_df_clean["dropoff_latitude"] != 0)]

train_df_clean = train_df_clean[(train_df_clean["pickup_longitude"] <= 180)]
train_df_clean = train_df_clean[(train_df_clean["pickup_latitude"] <= 180)]
train_df_clean = train_df_clean[(train_df_clean["dropoff_longitude"] <= 180)]
train_df_clean = train_df_clean[(train_df_clean["dropoff_latitude"] <= 180)]

train_df_clean = train_df_clean[(train_df_clean["pickup_longitude"] >= -180)]
train_df_clean = train_df_clean[(train_df_clean["pickup_latitude"] >= -180)]
train_df_clean = train_df_clean[(train_df_clean["dropoff_longitude"] >= -180)]
train_df_clean = train_df_clean[(train_df_clean["dropoff_latitude"] >= -180)]

train_df_clean.dropna(inplace=True)
train_df_clean.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-06-21 21:46:34.0000001,22.54,2015-06-21 21:46:34 UTC,-74.010483,40.717667,-73.985771,40.660366,1
1,2015-01-11 03:15:38.0000001,58.00,2015-01-11 03:15:38 UTC,-73.983330,40.738720,-73.933197,40.847225,1
2,2015-03-12 08:08:38.0000001,4.50,2015-03-12 08:08:38 UTC,-73.990173,40.756447,-73.985619,40.762829,1
3,2015-04-12 10:19:05.0000001,11.50,2015-04-12 10:19:05 UTC,-73.979279,40.723438,-74.004608,40.746948,6
4,2015-05-12 23:51:42.0000007,5.00,2015-05-12 23:51:42 UTC,-73.954796,40.779335,-73.944931,40.780087,1


## TRAIN. Jour / Nuit

In [8]:
train_df_clean["hour"] = train_df_clean["key"].str[10:13]
train_df_clean['hour'] = train_df_clean.hour.astype(np.int64)
# train_df_clean["day&night"] = 3
train_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3780169 entries, 0 to 3856808
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   key                object 
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   passenger_count    int64  
 8   hour               int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 288.4+ MB


In [9]:
train_df_clean.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour
0,2015-06-21 21:46:34.0000001,22.54,2015-06-21 21:46:34 UTC,-74.010483,40.717667,-73.985771,40.660366,1,21
1,2015-01-11 03:15:38.0000001,58.00,2015-01-11 03:15:38 UTC,-73.983330,40.738720,-73.933197,40.847225,1,3
2,2015-03-12 08:08:38.0000001,4.50,2015-03-12 08:08:38 UTC,-73.990173,40.756447,-73.985619,40.762829,1,8
3,2015-04-12 10:19:05.0000001,11.50,2015-04-12 10:19:05 UTC,-73.979279,40.723438,-74.004608,40.746948,6,10
4,2015-05-12 23:51:42.0000007,5.00,2015-05-12 23:51:42 UTC,-73.954796,40.779335,-73.944931,40.780087,1,23


In [25]:
train_df_clean["day&night"] = 1

train_df_clean["day&night"] = np.where(train_df_clean.hour >= 6,0,train_df_clean["day&night"])
train_df_clean["day&night"] = np.where(train_df_clean.hour >= 22,1,train_df_clean["day&night"])

In [26]:
train_df_clean.head(40)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour,day&night
0,2015-06-21 21:46:34.0000001,22.54,2015-06-21 21:46:34 UTC,-74.010483,40.717667,-73.985771,40.660366,1,21,0
1,2015-01-11 03:15:38.0000001,58.00,2015-01-11 03:15:38 UTC,-73.983330,40.738720,-73.933197,40.847225,1,3,1
2,2015-03-12 08:08:38.0000001,4.50,2015-03-12 08:08:38 UTC,-73.990173,40.756447,-73.985619,40.762829,1,8,0
3,2015-04-12 10:19:05.0000001,11.50,2015-04-12 10:19:05 UTC,-73.979279,40.723438,-74.004608,40.746948,6,10,0
4,2015-05-12 23:51:42.0000007,5.00,2015-05-12 23:51:42 UTC,-73.954796,40.779335,-73.944931,40.780087,1,23,1
5,2015-03-04 14:55:40.0000006,5.00,2015-03-04 14:55:40 UTC,-73.988747,40.744953,-73.995682,40.744221,2,14,0
6,2015-04-03 16:21:30.0000005,11.00,2015-04-03 16:21:30 UTC,-73.981056,40.774265,-73.959045,40.780674,2,16,0
7,2015-05-17 08:02:59.0000002,14.00,2015-05-17 08:02:59 UTC,-73.979210,40.723682,-74.017464,40.711395,1,8,0
8,2015-05-31 07:33:18.0000002,16.50,2015-05-31 07:33:18 UTC,-73.996986,40.755428,-73.953194,40.802101,3,7,0
9,2015-02-15 22:41:32.0000003,9.00,2015-02-15 22:41:32 UTC,-74.003448,40.733101,-74.008003,40.707214,1,22,1


## TRAIN. Calcul de la distance

In [ ]:
train_df_clean["distance"] = compute_distance(train_df_clean["pickup_latitude"],train_df_clean["pickup_longitude"],train_df_clean["dropoff_latitude"],train_df_clean["dropoff_longitude"])

In [ ]:
train_df_clean

In [ ]:
train_df_clean = train_df_clean[(train_df_clean["distance"] > 0)]
train_df_clean = train_df_clean.drop(columns=["key","pickup_datetime","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"])
train_df_clean.head()

## TRAIN. Visualisation avant Forest Isolation

In [ ]:
df_percent = train_df_clean.sample(frac=0.2)
boxplot = df_percent.boxplot(column=['fare_amount'])

In [ ]:
boxplot = df_percent.boxplot(column=['passenger_count'])

In [ ]:
boxplot = df_percent.boxplot(column=['distance'])

In [ ]:
boxplot = df_percent.boxplot(column=['heure'])

## TRAIN. Forest isolation

In [ ]:
model = IsolationForest(n_estimators = 100, max_samples = "auto", contamination = "auto", max_features = 1)
model.fit(train_df_clean)
train_df_clean["anomaly"] = model.predict(train_df_clean)
train_df_clean.head(20)

In [ ]:
anomaly = train_df_clean.loc[train_df_clean["anomaly"] == -1]
anomaly_index = list(anomaly.index)
print(anomaly)

In [ ]:
print(len(anomaly))

In [ ]:
train_df_clean = train_df_clean[train_df_clean["anomaly"]!= -1]
train_df_clean = train_df_clean.drop(columns=["anomaly"])

## TRAIN. Visualisation après Forest Isolation

In [ ]:
df_percent = train_df_clean.sample(frac=0.2)
boxplot = df_percent.boxplot(column=['fare_amount'])

In [ ]:
boxplot = df_percent.boxplot(column=['passenger_count'])

In [ ]:
boxplot = df_percent.boxplot(column=['distance'])

In [ ]:
boxplot = df_percent.boxplot(column=['heure'])

In [ ]:
matrice_correlation = train_df_clean.corr().round(3)
sns.heatmap(data = matrice_correlation, annot = True)

## TRAIN. Création des modèles et de leurs évaluations

In [ ]:
X = pd.DataFrame(train_df_clean, columns=["passenger_count","distance","heure"])
y = pd.Series(train_df_clean["fare_amount"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)           # Séparation du DF train en train et test

In [ ]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)

start_time = time.time()
regressor.fit(X_train, y_train)                                     # Entrainement
y_pred_XGB = regressor.predict(X_test)                              # Prediction
end_time = time.time()
print(f"MODEL XGBOOST --- {start_time - end_time} seconds ---")
print(mean_squared_error(y_test, y_pred_XGB))                       # Perte de régression par erreur quadratique moyenne.
print(r2_score(y_test, y_pred_XGB))                                 # Score

In [ ]:
lmodellineaire = LinearRegression()
start_time = time.time()
lmodellineaire.fit(X_train, y_train)
y_pred_SKL = lmodellineaire.predict(X_test)
end_time = time.time()
print(f"MODEL SKL --- {start_time - end_time} seconds ---")
print(mean_squared_error(y_test, y_pred_SKL))
print(r2_score(y_test, y_pred_SKL))

In [ ]:
random_forest = RandomForestRegressor() 
start_time = time.time()
random_forest.fit(X_train, y_train)
y_pred_RF = random_forest.predict(X_test)
end_time = time.time()
print(f"MODEL RF --- {start_time - end_time} seconds ---")
print(mean_squared_error(y_test, y_pred_RF))
print(r2_score(y_test, y_pred_RF))

## TRAIN. Conculsion

*On observant mes modèles je m'appercoit que le modèle XGBOOST est plus rapide à s'effectuer tout en étant plus précis au niveau de son score.*
*Je décide donc de prendre ce modèle pour les valeurs manquantes de mon fichier TEST*

## TEST. Nettoyage des données 

In [ ]:
test_df_clean = test_df[(test_df["passenger_count"] > 0)]
test_df_clean = test_df_clean[(test_df_clean["passenger_count"] < 7)]

test_df_clean = test_df_clean[(test_df_clean["pickup_longitude"] != 0)]
test_df_clean = test_df_clean[(test_df_clean["pickup_latitude"] != 0)]
test_df_clean = test_df_clean[(test_df_clean["dropoff_longitude"] != 0)]
test_df_clean = test_df_clean[(test_df_clean["dropoff_latitude"] != 0)]

test_df_clean = test_df_clean[(test_df_clean["pickup_longitude"] <= 180)]
test_df_clean = test_df_clean[(test_df_clean["pickup_latitude"] <= 180)]
test_df_clean = test_df_clean[(test_df_clean["dropoff_longitude"] <= 180)]
test_df_clean = test_df_clean[(test_df_clean["dropoff_latitude"] <= 180)]

test_df_clean = test_df_clean[(test_df_clean["pickup_longitude"] >= -180)]
test_df_clean = test_df_clean[(test_df_clean["pickup_latitude"] >= -180)]
test_df_clean = test_df_clean[(test_df_clean["dropoff_longitude"] >= -180)]
test_df_clean = test_df_clean[(test_df_clean["dropoff_latitude"] >= -180)]

test_df_clean.dropna(inplace=True)
test_df_clean.head()

## TEST. Calcul de la distance

In [ ]:
test_df["distance"] = compute_distance(global_test_df["pickup_latitude"],global_test_df["pickup_longitude"],global_test_df["dropoff_latitude"],global_test_df["dropoff_longitude"])
test_df_clean = test_df[(test_df["distance"] > 0)]
test_df_clean = test_df.drop(columns=["key","pickup_datetime","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"])

## TEST. Forest isolation

In [ ]:
model = IsolationForest(n_estimators = 100, max_samples = "auto", contamination = "auto", max_features = 1)
model.fit(test_df_clean)
test_df_clean["anomaly"] = model.predict(test_df_clean)
anomaly = test_df_clean.loc[test_df_clean["anomaly"] == -1]
anomaly_index = list(anomaly.index)
print(anomaly)
test_df_clean = test_df_clean[test_df_clean["anomaly"]!= -1]
test_df_clean = test_df_clean.drop(columns=["anomaly"])

## TEST. Prédictions "FARE_AMOUT"

In [ ]:
test_df_clean["fare_amount"] = regressor.predict(test_df_clean)

In [ ]:
test_df_clean.head(30)